In [ ]:
import pandas as pd
import seaborn as sns 
import numpy as np 
import matplotlib.pyplot as plt
from unidecode import unidecode
import re

In [ ]:
df = pd.read_excel('CarteirasTotais-DNC_V2.xlsx')

In [ ]:
df.head()

Remoção dos acentos das colunas e alteração dos caracteres diferentes de alfa numérico para "_"

In [ ]:
newColumnsName = []
for column_name in df:
  newColumnsName.append(re.sub('[^A-Za-z0-9]+', '_', unidecode(column_name).lower()))
df.columns = newColumnsName

Criação da coluna churn, e definição do seu valor com base nos valores da coluna upsale_downsale (churn = 1, demais valores = 0)

In [ ]:
df["upsale_downsale"].replace({"Churn": "churn", "Upsell": "upsell", "Downsell": "downsell", "Ok":"ok"}, inplace=True)
df['churn'] = df['upsale_downsale']
df['churn'].replace({"ok": '0', "upsell": '0', "downsell": '0', "churn": '1'}, inplace=True)

Remoção das colunas não interessantes

In [ ]:
df = df.drop(columns=['id_crm', 'id_navplat', 'valor_hoje', 'upsale_downsale', 'executivo_carteira_atual', 'leads_form', 'leads_ver_dato', 'total_de_listings', 'faixa_listings'])

Gráfico de correlação das variáveis permanecentes (verificar para alterar a coluna churn para inteiro, e então utilizar no gráfico)

In [ ]:
correlation = df.corr()

fig, ax = plt.subplots(figsize=(15,15)) 
plot = sns.heatmap(correlation, annot = True, ax=ax , linewidths=2)
plot

In [ ]:
df.head()

Remoção de colunas cuja correlação entre si é muito alta (somente deixando uma delas)

* valor_mensal e faturamento - 0.95
* utilizado_ofertas_simples e total_utlizado - 0.96
* total_contratado e contratado_ofertas_simples - 1
* contratado_destaques e utilizado_destaques - 0.96

Obs.: A correlação entre contratado super destaque e Leads é bem alta. Mas a principio entendi que deve ser mantido mas é bom avaliar questao com ImóvelWeb

In [ ]:
df.columns

In [ ]:
df = df.drop(columns=['valor_mensal','utilizado_destaque','utilizado_ofertas_simples','contratado_ofertas_simples'])

In [ ]:
df.head()

In [ ]:
df.dtypes

obs. as colunas com valores numéricos não necessitam formatação

obs2. a principio irei manter a coluna mês com a formatação que está por motivos de pouco tempo

Drop na coluna 'Status final' por ser irrelevante ao negócio sendo somente importante a informção de churn e não churn

In [ ]:
df = df.drop(columns=['status_final'])

Tentativa de tratar planilha de quality score - ignorar pelo momento

In [ ]:
df.isnull().sum()


# Tratamento de nulos

Contratado Freemiun -> Segundo o Sponser, os nulos significam não contratados : Portanto é possível tranformar os nulos em zero 

Total_utilizado -> Verificar com sponser (ao meu entender valores nulos significam que nenhum anuncio foi utilizado, portanto faz sentido substituir por zero)

obs. primeiro verificarei se a coluna regiao é relevante, se for por ser uma pequena porcentagem de nulos pode-se dropar somente as linhas

In [ ]:
df['contratado_freemium'] = df['contratado_freemium'].fillna(0)
df['utilizado_super_destaques'] = df['utilizado_super_destaques'].fillna(0)
df['utilizado_freemium'] = df['utilizado_freemium'].fillna(0)
df['total_utilizado'] = df['total_utilizado'].fillna(0)


In [ ]:
df.isnull().sum()

# Verificando relevância de features categoricas

obs. deveriamos fazer por mês, por falta de tempo farei apenas do total

In [ ]:
df['churn'].replace({'0': 0, '1': 1}, inplace=True)

In [ ]:
df.dtypes

In [ ]:
df['regiao'].unique()


Como não há constancia nos de região é melhor desconsiderar

In [ ]:
df = df.drop(columns=['regiao','cidade','bairro'])

In [ ]:
df.columns

#Oficina

In [ ]:
df['oficina'].unique()

In [ ]:
df['oficina'].replace({'wi': 'WI'}, inplace=True) # Alterando o valor wi para igualar a WI
df['oficina'].unique()

Faturamento

In [ ]:
df['frequencia_de_faturamento'].unique()

In [ ]:
#corrigir diferenças de nomes
df['frequencia_de_faturamento'] = df['frequencia_de_faturamento'].str.lower()
df['frequencia_de_faturamento'].unique()

In [ ]:
#remover espaços

df['frequencia_de_faturamento'] = df['frequencia_de_faturamento'].str.strip()
df['frequencia_de_faturamento'].unique()

In [ ]:
#renomeando unica vez

df['frequencia_de_faturamento'].replace({'única vez': 'unica_vez'}, inplace=True)
df['frequencia_de_faturamento'].unique()

Tipo de Plano

In [ ]:
df['tipo_de_plano'].unique()

Midia Ativa

In [ ]:
df['possui_midia_ativa_'].unique()

PF_PJ

In [ ]:
df['pf_pj'].unique()

Equipe

In [ ]:
df['equipe'].unique()

In [ ]:
df['equipe'].replace({'Relacionamento': 'RELACIONAMENTO','Jumbo':'JUMBO','Resellers':'RESELLERS'}, inplace=True) # Alterando para valores ficarem iguais
df['equipe'].unique()

One-hot-enconding

In [ ]:
#Variáveis Binárias para passar por one-hot-enconding

ohe_1 = ['oficina','tipo_de_plano','possui_midia_ativa_','pf_pj']

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

for col in ohe_1:
  df[col] = label_encoder.fit_transform(df[col])

df.head()


In [ ]:
#2. Transformar as variáveis não binárias

df =  pd.get_dummies(df, columns= ['equipe'])
df.head()

In [ ]:
df =  pd.get_dummies(df, columns= ['frequencia_de_faturamento'])
df.head()

In [ ]:
df['churn'].value_counts(normalize=True)

In [ ]:
df.shape[1]